# 第五章　顧客の退会を予測する
**決定木**<br>わかりやすく原因の分析を行うことができるのでビジネスでよく用いられる．

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
customer = pd.read_csv('Data100/customer.csv')
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47


In [4]:
uselog_month = pd.read_csv('Data100/use_log_month.csv')
uselog_month.head()

,month,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


In [6]:
# -- 過去一ヶ月の利用回数 --

# year_month:uselog_monthのユニーク
year_month = list(uselog_month['month'].unique())
predict_data = pd.DataFrame()

# 1〜year_monthの個数
for i in range(1, len(year_month)):
    
    # tmp：uselog_monthのmonthとi番目のyear_monthが一緒のuselog_month.
    # 月ごとにデータ入れてるってことかな
    tmp = uselog_month.loc[uselog_month['month']==year_month[i]]
    tmp.rename(columns={'count': 'count_pred'}, inplace=True)
    
    
    tmp_before = uselog_month.loc[uselog_month['month']==year_month[i-1]]
        
    del tmp_before['month']
    tmp_before.rename(columns={'count': 'count_{}'.format('1')}, inplace=True)
    tmp = tmp.merge(tmp_before, on='customer_id', how='left')
    predict_data = pd.concat([predict_data, tmp], ignore_index=True)
predict_data.head()

,month,customer_id,count_pred,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


In [13]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer['is_deleted']==1]
exit_customer['exit_date']  = None
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])

for i in range(len(exit_customer)):
    exit_customer['exit_date'].iloc[i] = exit_customer['end_date'].iloc[i] -relativedelta(months=1)
exit_customer['month'] = exit_customer['exit_date'].dt.strftime('%Y%m')
predict_data['month'] = predict_data['month'].astype(str)
exit_uselog = predict_data.merge(exit_customer, on=['customer_id', 'month'], how='left')
print(len(predict_data))
exit_uselog.head()    

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

33851


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,month,customer_id,count_pred,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
exit_uselog = exit_uselog.dropna(subset=['name'])
exit_uselog.head()

,month,customer_id,count_pred,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


In [17]:
continue_customer = customer.loc[customer['is_deleted']==0]
continue_uselog = predict_data.merge(continue_customer, on=['customer_id'], how='left')
continue_uselog = continue_uselog.dropna(subset=['name'])
print(len(continue_uselog))
continue_uselog.head()

27422


,month,customer_id,count_pred,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201805,AS002855,5,4.0,XXXX,C03,F,2016-11-01,NaN,CA1,...,ナイト,6000.0,通常,4.500000,5.0,7.0,2.0,1.0,2019-04-30,29.0
1,201805,AS009373,4,3.0,XX,C01,F,2015-11-01,NaN,CA1,...,オールタイム,10500.0,通常,5.083333,5.0,7.0,3.0,1.0,2019-04-30,41.0
2,201805,AS015233,7,NaN,XXXXX,C01,M,2018-05-13,NaN,CA2,...,オールタイム,10500.0,入会費半額,7.545455,7.0,11.0,4.0,1.0,2019-04-30,11.0
3,201805,AS015315,3,6.0,XXXXX,C01,M,2015-07-01,NaN,CA1,...,オールタイム,10500.0,通常,4.833333,5.0,7.0,3.0,1.0,2019-04-30,45.0
4,201805,AS015739,5,7.0,XXXXX,C03,M,2017-06-01,NaN,CA1,...,ナイト,6000.0,通常,5.583333,5.5,8.0,4.0,1.0,2019-04-30,22.0


In [20]:
continue_uselog = continue_uselog.sample(frac=1).reset_index(drop=True)
continue_uselog = continue_uselog.drop_duplicates(subset="customer_id")
print(len(continue_uselog))
continue_uselog.head()

2842


,month,customer_id,count_pred,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201903,HI784547,9,7.0,XXXXX,C01,F,2018-12-09,NaN,CA3,...,オールタイム,10500.0,入会費無料,8.000000,8.0,9.0,7.0,1.0,2019-04-30,4.0
1,201903,IK438932,6,5.0,XXXX,C01,M,2016-08-01,NaN,CA1,...,オールタイム,10500.0,通常,4.916667,5.0,7.0,3.0,1.0,2019-04-30,32.0
2,201901,PL240187,9,7.0,XXXXX,C02,F,2018-11-15,NaN,CA1,...,デイタイム,7500.0,通常,8.000000,8.0,9.0,7.0,1.0,2019-04-30,5.0
3,201808,IK180880,7,6.0,XXX,C02,F,2017-12-01,NaN,CA3,...,デイタイム,7500.0,入会費無料,6.833333,7.0,10.0,4.0,1.0,2019-04-30,16.0
4,201809,PL918865,9,8.0,XXXXX,C01,M,2018-01-01,NaN,CA1,...,オールタイム,10500.0,通常,7.083333,7.0,10.0,4.0,1.0,2019-04-30,15.0


In [21]:
predict_data = pd.concat([continue_uselog, exit_uselog],ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,calc_date,campaign_id,campaign_name,class,class_name,count_1,count_pred,customer_id,end_date,exit_date,...,max,mean,median,membership_period,min,month,name,price,routine_flg,start_date
0,2019-04-30,CA3,入会費無料,C01,オールタイム,7.0,9,HI784547,NaN,NaN,...,9.0,8.000000,8.0,4.0,7.0,201903,XXXXX,10500.0,1.0,2018-12-09
1,2019-04-30,CA1,通常,C01,オールタイム,5.0,6,IK438932,NaN,NaN,...,7.0,4.916667,5.0,32.0,3.0,201903,XXXX,10500.0,1.0,2016-08-01
2,2019-04-30,CA1,通常,C02,デイタイム,7.0,9,PL240187,NaN,NaN,...,9.0,8.000000,8.0,5.0,7.0,201901,XXXXX,7500.0,1.0,2018-11-15
3,2019-04-30,CA3,入会費無料,C02,デイタイム,6.0,7,IK180880,NaN,NaN,...,10.0,6.833333,7.0,16.0,4.0,201808,XXX,7500.0,1.0,2017-12-01
4,2019-04-30,CA1,通常,C01,オールタイム,8.0,9,PL918865,NaN,NaN,...,10.0,7.083333,7.0,15.0,4.0,201809,XXXXX,10500.0,1.0,2018-01-01


In [23]:
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["month"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)
predict_data.head()


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,calc_date,campaign_id,campaign_name,class,class_name,count_1,count_pred,customer_id,end_date,exit_date,...,median,membership_period,min,month,name,price,routine_flg,start_date,period,now_date
0,2019-04-30,CA3,入会費無料,C01,オールタイム,7.0,9,HI784547,NaN,NaN,...,8.0,4.0,7.0,201903,XXXXX,10500.0,1.0,2018-12-09,2,2019-03-01
1,2019-04-30,CA1,通常,C01,オールタイム,5.0,6,IK438932,NaN,NaN,...,5.0,32.0,3.0,201903,XXXX,10500.0,1.0,2016-08-01,31,2019-03-01
2,2019-04-30,CA1,通常,C02,デイタイム,7.0,9,PL240187,NaN,NaN,...,8.0,5.0,7.0,201901,XXXXX,7500.0,1.0,2018-11-15,1,2019-01-01
3,2019-04-30,CA3,入会費無料,C02,デイタイム,6.0,7,IK180880,NaN,NaN,...,7.0,16.0,4.0,201808,XXX,7500.0,1.0,2017-12-01,8,2018-08-01
4,2019-04-30,CA1,通常,C01,オールタイム,8.0,9,PL918865,NaN,NaN,...,7.0,15.0,4.0,201809,XXXXX,10500.0,1.0,2018-01-01,8,2018-09-01


In [24]:
predict_data.isna().sum()

calc_date               0
campaign_id             0
campaign_name           0
class                   0
class_name              0
count_1               245
count_pred              0
customer_id             0
end_date             2842
exit_date            2842
gender                  0
is_deleted              0
max                     0
mean                    0
median                  0
membership_period       0
min                     0
month                   0
name                    0
price                   0
routine_flg             0
start_date              0
period                  0
now_date                0
dtype: int64

In [25]:
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

calc_date               0
campaign_id             0
campaign_name           0
class                   0
class_name              0
count_1                 0
count_pred              0
customer_id             0
end_date             2649
exit_date            2649
gender                  0
is_deleted              0
max                     0
mean                    0
median                  0
membership_period       0
min                     0
month                   0
name                    0
price                   0
routine_flg             0
start_date              0
period                  0
now_date                0
dtype: int64

## ダミー変数を作る

In [26]:
target_col = ['campaign_name', 'class_name', 'gender', 'count_1', 'routine_flg', 'period', 'is_deleted']
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,入会費無料,オールタイム,F,7.0,1.0,2,0.0
1,通常,オールタイム,M,5.0,1.0,31,0.0
2,通常,デイタイム,F,7.0,1.0,1,0.0
3,入会費無料,デイタイム,F,6.0,1.0,8,0.0
4,通常,オールタイム,M,8.0,1.0,8,0.0


In [27]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,7.0,1.0,2,0.0,0,1,0,1,0,0,1,0
1,5.0,1.0,31,0.0,0,0,1,1,0,0,0,1
2,7.0,1.0,1,0.0,0,0,1,0,1,0,1,0
3,6.0,1.0,8,0.0,0,1,0,0,1,0,1,0
4,8.0,1.0,8,0.0,0,0,1,1,0,0,0,1


**いらないダミー変数は消す**

In [30]:
predict_data.drop(['campaign_name_入会費半額', 'gender_M', 'class_name_ナイト'], axis=1, inplace=True)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,gender_F
0,7.0,1.0,2,0.0,1,0,1,0,1
1,5.0,1.0,31,0.0,0,1,1,0,0
2,7.0,1.0,1,0.0,0,1,0,1,1
3,6.0,1.0,8,0.0,1,0,0,1,1
4,8.0,1.0,8,0.0,0,1,1,0,0


## 決定木を用いて退会予測モデルを作成

In [31]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data['is_deleted'] == 1]
conti = predict_data.loc[predict_data['is_deleted'] == 0]

X = pd.concat([exit, conti], ignore_index=True)
y = X['is_deleted']
del X['is_deleted']

X.head()

,count_1,routine_flg,period,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,gender_F
0,3.0,0.0,2,0,1,1,0,0
1,3.0,1.0,6,0,1,1,0,0
2,3.0,0.0,11,0,1,1,0,0
3,3.0,0.0,3,0,1,1,0,0
4,4.0,0.0,8,0,1,0,0,1


In [32]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

In [33]:
model = DecisionTreeClassifier(random_state=0, max_depth=4)
model.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best')

In [35]:
results_test = pd.DataFrame(model.predict(X_test))
results_test.head()

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,1.0


In [36]:
model.score(X_test, y_test)

0.9416846652267818

In [37]:
model.score(X_train, y_train)

0.9387387387387387

In [38]:
model.feature_importances_

array([0.38359378, 0.46383164, 0.15029536, 0.        , 0.00105256,
       0.        , 0.        , 0.00122666])

In [39]:
y_pred = model.predict(X_test)

In [41]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96       655
         1.0       0.95      0.85      0.89       271

    accuracy                           0.94       926
   macro avg       0.94      0.91      0.93       926
weighted avg       0.94      0.94      0.94       926

